## Create xml for training images

In [98]:
import xml.etree.cElementTree as ET
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from detecto import core, utils, visualize

In [23]:
## read training csv file
train = pd.read_csv("C:\\Users\\kohsi\\Desktop\\wheat_detection\\global-wheat-detection\\train.csv")
train.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [26]:
print(train.width.unique())
print(train.height.unique())

[1024]
[1024]


- All images are 1024 x 1024


In [38]:
# calculate xmax, ymax
bboxs = [ bbox[1:-1].split(', ') for bbox in train['bbox']]
bboxs = [ f"{int(float(bbox[0]))},{int(float(bbox[1]))},{int(float(bbox[0]))+int(float(bbox[2]))},
         {int(float(bbox[1])) + int(float(bbox[3]))},wheat" for bbox in bboxs]
train['bbox_'] = bboxs
train.head()

,image_id,width,height,bbox,source,bbox_
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1,"834,222,890,258,wheat"
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1,"226,548,356,606,wheat"
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1,"377,504,451,664,wheat"
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1,"834,95,943,202,wheat"
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1,"26,144,150,261,wheat"


In [91]:
# for each unique image, create a dictionary of {image:bbox}
uid = train['image_id'].unique()

image_bbox_dict = {}

for image_id in (uid):
    annt_list = train.loc[train['image_id']== iid, 'bbox_'].values.tolist()
    image_bbox_dict[image_id] = annt_list
    
image_bbox_dict

{'b6ab77fd7': ['273,284,386,376,wheat',
  '494,125,599,217,wheat',
  '412,220,527,317,wheat',
  '332,404,463,491,wheat',
  '289,742,376,814,wheat',
  '775,586,847,642,wheat',
  '389,927,569,1024,wheat',
  '673,337,752,432,wheat',
  '911,243,1021,322,wheat',
  '668,81,778,151,wheat',
  '716,7,796,71,wheat',
  '217,865,332,967,wheat',
  '916,46,980,102,wheat',
  '852,640,934,734,wheat',
  '64,619,148,714,wheat',
  '292,549,399,631,wheat',
  '134,228,275,299,wheat',
  '430,13,614,92,wheat',
  '875,740,969,801,wheat'],
 'b53afdf5c': ['273,284,386,376,wheat',
  '494,125,599,217,wheat',
  '412,220,527,317,wheat',
  '332,404,463,491,wheat',
  '289,742,376,814,wheat',
  '775,586,847,642,wheat',
  '389,927,569,1024,wheat',
  '673,337,752,432,wheat',
  '911,243,1021,322,wheat',
  '668,81,778,151,wheat',
  '716,7,796,71,wheat',
  '217,865,332,967,wheat',
  '916,46,980,102,wheat',
  '852,640,934,734,wheat',
  '64,619,148,714,wheat',
  '292,549,399,631,wheat',
  '134,228,275,299,wheat',
  '430,13,6

In [92]:
len(image_bbox_dict)

3373

In [12]:
# function to indent xml properly
def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [77]:
def create_labimg_xml(image_path, annotation_list):

    image_path = Path(image_path)
    img = np.array(Image.open(image_path).convert('RGB'))

    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = str(image_path.parent.name)
    ET.SubElement(annotation, 'filename').text = str(image_path.name)
    ET.SubElement(annotation, 'path').text = str(image_path)

    source = ET.SubElement(annotation, 'source')
    ET.SubElement(source, 'database').text = 'Unknown'

    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = str (img.shape[1])
    ET.SubElement(size, 'height').text = str(img.shape[0])
    ET.SubElement(size, 'depth').text = str(img.shape[2])

    ET.SubElement(annotation, 'segmented').text = '0'

    for annot in annotation_list:
        tmp_annot = annot.split(',')
        cords, label = tmp_annot[0:-1], tmp_annot[-1]
        xmin, ymin, xmax, ymax = cords[0], cords[1], cords[2], cords[3]

        object = ET.SubElement(annotation, 'object')
        ET.SubElement(object, 'name').text = label
        ET.SubElement(object, 'pose').text = 'Unspecified'
        ET.SubElement(object, 'truncated').text = '0'
        ET.SubElement(object, 'difficult').text = '0'

        bndbox = ET.SubElement(object, 'bndbox')
        ET.SubElement(bndbox, 'xmin').text = str(xmin)
        ET.SubElement(bndbox, 'ymin').text = str(ymin)
        ET.SubElement(bndbox, 'xmax').text = str(xmax)
        ET.SubElement(bndbox, 'ymax').text = str(ymax)

    indent(annotation)
    tree = ET.ElementTree(annotation)
    xml_file_name = image_path.parent / (image_path.name.split('.')[0]+'.xml')
    tree.write(xml_file_name)



In [97]:
for key in image_bbox_dict:
    anotation_list = image_bbox_dict[key]
    print('INFO: processing ' + key + '.jpg')
    create_labimg_xml('C:/Users/kohsi/Desktop/wheat_detection/global-wheat-detection/train/' + key + '.jpg', anotation_list)
    

INFO: processing b6ab77fd7.jpg
INFO: processing b53afdf5c.jpg
INFO: processing 7b72ea0fb.jpg
INFO: processing 91c9d9c38.jpg
INFO: processing 41c0123cc.jpg
INFO: processing c682d8c66.jpg
INFO: processing e46378032.jpg
INFO: processing 71fd03d51.jpg
INFO: processing 83a5d8ac8.jpg
INFO: processing 02b6f199c.jpg
INFO: processing 1bda74f14.jpg
INFO: processing 3e1e66c76.jpg
INFO: processing 4a3f67d2e.jpg
INFO: processing 2ccec6dcc.jpg
INFO: processing 0435654cd.jpg
INFO: processing 372d12c55.jpg
INFO: processing f5093f57d.jpg
INFO: processing c6b4933f7.jpg
INFO: processing 895420d9f.jpg
INFO: processing 8d6bb70e5.jpg
INFO: processing 531acb360.jpg
INFO: processing 4aeb37788.jpg
INFO: processing afb7671e5.jpg
INFO: processing cc9092474.jpg
INFO: processing c3c03e0b7.jpg
INFO: processing 42e247347.jpg
INFO: processing 5738fcf14.jpg
INFO: processing 31a6e55f2.jpg
INFO: processing ca4cb597b.jpg
INFO: processing 4e5c1078d.jpg
INFO: processing 29e44e305.jpg
INFO: processing 43e3ccd92.jpg
INFO: pr

INFO: processing 9d8a3a164.jpg
INFO: processing 1fe63a46a.jpg
INFO: processing c07795af6.jpg
INFO: processing 24dda66d6.jpg
INFO: processing 2dbaac896.jpg
INFO: processing 546a0e9fc.jpg
INFO: processing d7612c04a.jpg
INFO: processing eb3cffc94.jpg
INFO: processing 84d4ca3d3.jpg
INFO: processing 70cfaefe9.jpg
INFO: processing 2041a2d49.jpg
INFO: processing ab026459d.jpg
INFO: processing 2fc0fadfa.jpg
INFO: processing 2d159a876.jpg
INFO: processing 1f9a8abcd.jpg
INFO: processing 8a7d9b4b0.jpg
INFO: processing fe0d6c114.jpg
INFO: processing aa10c1bab.jpg
INFO: processing b1341e5ff.jpg
INFO: processing 1908103f1.jpg
INFO: processing 8841bcb77.jpg
INFO: processing aeb9f155b.jpg
INFO: processing 5cda6a08c.jpg
INFO: processing b6a159fd4.jpg
INFO: processing 19fd72f8b.jpg
INFO: processing b95fd89e3.jpg
INFO: processing a4ac9094c.jpg
INFO: processing e9d060d22.jpg
INFO: processing a245ab7f9.jpg
INFO: processing 20ebdfc4b.jpg
INFO: processing 97851b9c6.jpg
INFO: processing f72cabcea.jpg
INFO: pr

INFO: processing 1c3032ee8.jpg
INFO: processing 9c0466c9f.jpg
INFO: processing 79a1d0dad.jpg
INFO: processing 5a0e2133c.jpg
INFO: processing a0f951ca5.jpg
INFO: processing 037024e13.jpg
INFO: processing 12e8267b0.jpg
INFO: processing 39ad9affa.jpg
INFO: processing 3d1c7f78a.jpg
INFO: processing 1478a43f1.jpg
INFO: processing 15c21964a.jpg
INFO: processing eba411885.jpg
INFO: processing ecad4b636.jpg
INFO: processing 45a0724bf.jpg
INFO: processing 34b5b6acb.jpg
INFO: processing 3a1c78a92.jpg
INFO: processing bb3a2e25d.jpg
INFO: processing 16d922704.jpg
INFO: processing f1aaca96f.jpg
INFO: processing f3fcfaa0d.jpg
INFO: processing 9983d297c.jpg
INFO: processing 3eeb41f2a.jpg
INFO: processing 9644dd8a1.jpg
INFO: processing e6e15309d.jpg
INFO: processing 58ce0381b.jpg
INFO: processing 5f9cfe156.jpg
INFO: processing 7c4a4a55c.jpg
INFO: processing 0d2e2b65b.jpg
INFO: processing 673b64e17.jpg
INFO: processing 8b342101d.jpg
INFO: processing 47e218059.jpg
INFO: processing 7f84d451f.jpg
INFO: pr

INFO: processing a40ab0738.jpg
INFO: processing 24fb4b33d.jpg
INFO: processing 5a61c3e15.jpg
INFO: processing 86b661ae0.jpg
INFO: processing a6c1cc190.jpg
INFO: processing 962ccb192.jpg
INFO: processing 46930c2f2.jpg
INFO: processing ed59f37d2.jpg
INFO: processing 7f4f977bf.jpg
INFO: processing 2d990708e.jpg
INFO: processing e9491d2d6.jpg
INFO: processing a5c5d1e9f.jpg
INFO: processing 2e867b62b.jpg
INFO: processing cabe03050.jpg
INFO: processing a70061d2b.jpg
INFO: processing fefd59985.jpg
INFO: processing d2ae1e503.jpg
INFO: processing 6f24b1bab.jpg
INFO: processing e4b256788.jpg
INFO: processing fe029eca4.jpg
INFO: processing 2e262da1c.jpg
INFO: processing ba290b964.jpg
INFO: processing 119969932.jpg
INFO: processing 4cfb86eff.jpg
INFO: processing 54668a040.jpg
INFO: processing c7d3b408d.jpg
INFO: processing 9d78cb5d0.jpg
INFO: processing 91c9c8b5e.jpg
INFO: processing 95d633618.jpg
INFO: processing 43f9b9fc5.jpg
INFO: processing 331a697b2.jpg
INFO: processing 7257fdd84.jpg
INFO: pr

INFO: processing fc8cf268b.jpg
INFO: processing fbecfeba8.jpg
INFO: processing ff1716939.jpg
INFO: processing 2cc49e1ba.jpg
INFO: processing 1ff3f11c4.jpg
INFO: processing b0833ce3d.jpg
INFO: processing 768a74e08.jpg
INFO: processing 8a5310dc4.jpg
INFO: processing 1708360c3.jpg
INFO: processing 5ccb447e6.jpg
INFO: processing 0b9624bd4.jpg
INFO: processing 666ca30b3.jpg
INFO: processing c6454073b.jpg
INFO: processing 4af27998f.jpg
INFO: processing f6d969b95.jpg
INFO: processing e1cff2554.jpg
INFO: processing 0923194d6.jpg
INFO: processing e4b7e3af6.jpg
INFO: processing 8f06b77e5.jpg
INFO: processing 6f10a6f8f.jpg
INFO: processing a13c89a7f.jpg
INFO: processing 2051f1446.jpg
INFO: processing 38dc44b85.jpg
INFO: processing 64b352e9a.jpg
INFO: processing 5ab074c28.jpg
INFO: processing 90677210d.jpg
INFO: processing efc5642bb.jpg
INFO: processing 4e652f7f1.jpg
INFO: processing aa29cef4a.jpg
INFO: processing 8b12ac747.jpg
INFO: processing 9ebdbabff.jpg
INFO: processing f5837f263.jpg
INFO: pr

...

INFO: processing 04fe56fd3.jpg
INFO: processing 644eff133.jpg
INFO: processing 57e21a852.jpg
INFO: processing 755becfe1.jpg
INFO: processing bc1666f8c.jpg
INFO: processing f1e0fb4d4.jpg
INFO: processing d94f9d8a2.jpg
INFO: processing 0f47f54f1.jpg
INFO: processing 85189f1bf.jpg
INFO: processing aadb83e1d.jpg
INFO: processing 7d0d1f2f4.jpg
INFO: processing 9561a71a0.jpg
INFO: processing f7fceadee.jpg
INFO: processing 0b7712f7a.jpg
INFO: processing 9bccf7e53.jpg
INFO: processing d4abcc41e.jpg
INFO: processing 02fe8c28e.jpg
INFO: processing 2b5e0f1a2.jpg
INFO: processing c64da3b0d.jpg
INFO: processing c9ade75be.jpg
INFO: processing 77222a135.jpg
INFO: processing 4ddaaf93f.jpg
INFO: processing b922186af.jpg
INFO: processing 2889b72ef.jpg
INFO: processing 346efc9f4.jpg
INFO: processing 15c794bfc.jpg
INFO: processing 4ce73a46c.jpg
INFO: processing a57808743.jpg
INFO: processing 723068e09.jpg
INFO: processing dbe68e1c1.jpg
INFO: processing f9f72d23c.jpg
INFO: processing 80e6b6670.jpg
INFO: pr

INFO: processing c94f8dd92.jpg
INFO: processing e290bc809.jpg
INFO: processing f3d0759c8.jpg
INFO: processing e142e85ad.jpg
INFO: processing 9c832432c.jpg
INFO: processing 22f341965.jpg
INFO: processing c0fa19bac.jpg
INFO: processing aa644920c.jpg
INFO: processing c0ad0793a.jpg
INFO: processing db3eb3f0c.jpg
INFO: processing 509a7a9b5.jpg
INFO: processing f21c0d164.jpg
INFO: processing 1b3380c60.jpg
INFO: processing 233cb8750.jpg
INFO: processing 81e06a4cd.jpg
INFO: processing e08c90ebe.jpg
INFO: processing ea72a0c0c.jpg
INFO: processing 74d747bcf.jpg
INFO: processing 27da826ad.jpg
INFO: processing 83aae58fa.jpg
INFO: processing dc7f9cee6.jpg
INFO: processing b6b899073.jpg
INFO: processing 63f63c857.jpg
INFO: processing 0dc9eb168.jpg
INFO: processing 78752f185.jpg
INFO: processing 034090252.jpg
INFO: processing a30ad4024.jpg
INFO: processing f05830c46.jpg
INFO: processing 31016e613.jpg
INFO: processing 2e0c19135.jpg
INFO: processing ccfff0629.jpg
INFO: processing 1914c04d6.jpg
INFO: pr

INFO: processing b5e726c10.jpg
INFO: processing 2ff4e8bb0.jpg
INFO: processing 30f54e611.jpg
INFO: processing a326e7d8e.jpg
INFO: processing a3bcd1d0f.jpg
INFO: processing 382bf4ed7.jpg
INFO: processing be589b720.jpg
INFO: processing 97372d371.jpg
INFO: processing bbce58f71.jpg
INFO: processing b45096c1e.jpg
INFO: processing b3c96d5ad.jpg
INFO: processing 1be796d9d.jpg
INFO: processing ba9133208.jpg
INFO: processing ac891ac35.jpg
INFO: processing a5904a10b.jpg
INFO: processing ea0e9e846.jpg
INFO: processing 92f3b252f.jpg
INFO: processing 59d43a53f.jpg
INFO: processing 290ae67be.jpg
INFO: processing 8afea785a.jpg
INFO: processing 44b7163ea.jpg
INFO: processing d1f73158a.jpg
INFO: processing fabaeac81.jpg
INFO: processing b57f67edf.jpg
INFO: processing c8defc0d1.jpg
INFO: processing 5f5181592.jpg
INFO: processing 38b19d4f5.jpg
INFO: processing f233cfd8b.jpg
INFO: processing 2391f283f.jpg
INFO: processing 07479da31.jpg
INFO: processing 5e182e37e.jpg
INFO: processing ce3999eb9.jpg
INFO: pr

INFO: processing 56c1bc7f0.jpg
INFO: processing 7f4d07cc0.jpg
INFO: processing eb69f5522.jpg
INFO: processing f355c9fe2.jpg
INFO: processing c4731ac9e.jpg
INFO: processing 91ef6ef2b.jpg
INFO: processing 8d4f8b1cc.jpg
INFO: processing 6b62a043a.jpg
INFO: processing 522edf5c3.jpg
INFO: processing 463a95dac.jpg
INFO: processing cf483de89.jpg
INFO: processing 452859b3a.jpg
INFO: processing 93d056da6.jpg
INFO: processing e46eadfe8.jpg
INFO: processing bf4a133c9.jpg
INFO: processing 5c2e16f27.jpg
INFO: processing fb6e04c0c.jpg
INFO: processing a024116dc.jpg
INFO: processing cc2bbb768.jpg
INFO: processing 848e992bd.jpg
INFO: processing 8f507b3f7.jpg
INFO: processing d557e8635.jpg
INFO: processing 2eaac7a41.jpg
INFO: processing 833577238.jpg
INFO: processing 4a654c127.jpg
INFO: processing 4972b9493.jpg
INFO: processing 2d1b6ce2a.jpg
INFO: processing 15153bb54.jpg
INFO: processing d54ece3f3.jpg
INFO: processing a8e1c8b37.jpg
INFO: processing 35b59e0a0.jpg
INFO: processing e30cd9270.jpg
INFO: pr

INFO: processing 6a614e72e.jpg
INFO: processing 4def855a8.jpg
INFO: processing 85e3ce681.jpg
INFO: processing 44afd181e.jpg
INFO: processing 528621931.jpg
INFO: processing a28a4ad8c.jpg
INFO: processing f9ece438e.jpg
INFO: processing 79ce67248.jpg
INFO: processing 277fa7280.jpg
INFO: processing f5607392f.jpg
INFO: processing 0ca410614.jpg
INFO: processing ac6b1b3a3.jpg
INFO: processing 96367fda5.jpg
INFO: processing 03448724b.jpg
INFO: processing 19e5ab184.jpg
INFO: processing 457752437.jpg
INFO: processing 2b32a4cc4.jpg
INFO: processing 059125072.jpg
INFO: processing 0114c88aa.jpg
INFO: processing 3753c58f7.jpg
INFO: processing 38b259a9e.jpg
INFO: processing 1b99d2fd6.jpg
INFO: processing 47b0d504c.jpg
INFO: processing 4112a34bd.jpg
INFO: processing 71d2a4b3c.jpg
INFO: processing 980c88da5.jpg
INFO: processing bd20bc5c9.jpg
INFO: processing 02992044c.jpg
INFO: processing b3822b5cd.jpg
INFO: processing 3f8f6b1a1.jpg
INFO: processing 398708128.jpg
INFO: processing d950bfe39.jpg
INFO: pr

## Data Augmentation

##  Train Model

In [ ]:
dataset = core.Dataset('C:/Users/kohsi/Desktop/wheat_detection/global-wheat-detection/train/')
model = core.Model(['wheat'])

model.fit(dataset)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\kohsi/.cache\torch\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100.0%
C:\Users\kohsi\Desktop\wheat_detection\wheat_venv\lib\site-packages\torch\nn\functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


In [ ]:
model.save('model_weights.pth')

## Predict for test images


In [ ]:
# Specify the path to your image
image = utils.read_image('C:/Users/kohsi/Desktop/wheat_detection/global-wheat-detection/test/2fd875eaa.jpg')
predictions = model.predict(image)

# predictions format: (labels, boxes, scores)
labels, boxes, scores = predictions

# ['alien', 'bat', 'bat']
print(labels) 

#           xmin       ymin       xmax       ymax
# tensor([[ 569.2125,  203.6702, 1003.4383,  658.1044],
#         [ 276.2478,  144.0074,  579.6044,  508.7444],
#         [ 277.2929,  162.6719,  627.9399,  511.9841]])
print(boxes)

# tensor([0.9952, 0.9837, 0.5153])
print(scores)

In [ ]:
visualize.show_labeled_image(image, boxes, labels)